In [105]:
import chordparser
from dataclasses import dataclass
from enum import Enum
import torch.nn as nn
from typing import List, Tuple

In [106]:
parser = chordparser.Parser()
c1 = parser.create_chord('C9')
print(c1)
print(f'root={dir(c1.root)}')

C9
root=['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_doubleflat', '_doublesharp', '_flat', '_flat_tuple', '_note_values', '_notes_tuple', '_sharp', '_sharp_tuple', '_symbol_signs', '_symbols', 'accidental', 'letter', 'letter_value', 'num_value', 'shift_l', 'shift_s', 'symbol', 'symbol_value', 'transpose', 'transpose_simple', 'value']


In [107]:
class NoteIndex(Enum):
    C = 0
    Csharp = 1
    D = 2
    Dsharp = 3
    E = 4
    F = 5
    Fsharp = 6
    G = 7
    Gsharp = 8
    A = 9
    Asharp = 10
    B = 11
    
    def to_chord_format(self):
        return self.name.replace('sharp', '#')


QUALITIES_DISPLAY_NAMES = {0: '', 1: 'm', 2: 'dim', 3: 'aug', 4: 'sus4', 5: 'sus2', 6: '5'}
class ChordQuality(Enum):
    major = 0
    dominant = 0
    minor = 1
    diminished = 2
    augmented = 3
    sus4 = 4
    sus2 = 5
    power = 6
    
    def to_chord_format(self):
        return QUALITIES_DISPLAY_NAMES[self.value]

EXTRAS_DISPLAY_NAMES = {0: '', 1: '7', 2: 'M', 3: '9', 4: '13'}
class ChordExtra(Enum):
    ExtraNone = 0
    seventh = 1
    major_seventh = 2
    ninth = 3
    thirteenth = 4

    def to_chord_format(self):
        return EXTRAS_DISPLAY_NAMES[self.value]

ADD_DISPLAY_NAMES = {0: '', 1: 'add2', 2: 'add4', 3: 'add6'}
class ChordAdd(Enum):
    AddNone = 0
    Add2 = 1
    Add4 = 2
    Add6 = 3
    
    def to_chord_format(self):
        return ADD_DISPLAY_NAMES[self.value]

@dataclass(frozen=True)
class EncodedChord:
    root: NoteIndex
    relative_bass: NoteIndex
    quality: ChordQuality
    extra: ChordExtra
    add: ChordAdd
    def to_tuple(v):
        return (v.root, v.relative_bass, v.quality.value, v.extra.value, v.add.value)
    
    def to_chord(v):
        root = NoteIndex(v.root).to_chord_format()
        if v.relative_bass:
            bass_index = (v.root + v.relative_bass) % 12
            bass = f'/{NoteIndex(bass_index).to_chord_format()}'
        else:
            bass = ''
        quality = v.quality.to_chord_format()
        extra = v.extra.to_chord_format()
        add = v.add.to_chord_format()
        result = f'{root}{quality}{extra}{add}{bass}'
        return result

class ChordEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.cp = chordparser.Parser()

    def _encode_chord_add(self, chord: chordparser.Chord) -> ChordAdd:
        if not chord.add:
            return ChordAdd.AddNone
        
        adds = chord.add
        if len(adds) > 1:
            raise ValueError(f'Chord {chord.string} has more than 1 adds: {adds}')
        
        # Enum name is Add{value}
        first_val = f'Add{adds[0][1]}'
        result = ChordAdd[first_val]
        return result
    
    def _encode_chord_quality(self, chord: chordparser.Chord) -> ChordQuality:
        if not chord.quality.value:
            raise ValueError(f'Chord {chord.string} has no quality.value: {chord.quality}')
        
        result = ChordQuality[chord.quality.value]
        return result

    def _encode_chord_extra(self, chord: chordparser.Chord) -> ChordExtra:
        extra_name = chord.quality.ext
        if not extra_name:
            return ChordExtra.ExtraNone
        
        extra_key_name = extra_name.replace(' ', '_')
        result = ChordExtra[extra_key_name]
        return result

    def forward(self, chord_raw: str) -> EncodedChord:
        chord = self.cp.create_chord(chord_raw)
        root_index = chord.root.num_value()
        bass_index = root_index if chord.bass is None else chord.bass.num_value()
        relative_bass = (bass_index - root_index) % 12
        quality = self._encode_chord_quality(chord)
        extra = self._encode_chord_extra(chord)
        add = self._encode_chord_add(chord)

        result = EncodedChord(root=root_index, relative_bass=relative_bass, quality=quality, extra=extra, add=add)
        return result
    
    
class ChordDecoder(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, encoded_chord: EncodedChord) -> str:
        result = encoded_chord.to_chord()
        return result
    

In [115]:
encoder = ChordEncoder()
print(encoder('Em7add6/D').to_tuple())
print(encoder('Em7add6/D').to_chord())

(4, 10, 1, 1, 3)
Em7add6/D


In [89]:
print(EncodedChord(4, 2, 1, 1, 3).to_chord())

AttributeError: 'int' object has no attribute 'to_chord_format'

In [69]:
ChordQuality(0).to_chord_format()

0


''